This project is an interactive crypto coin analysis tool that retrieves market and holder data from Coingecko API, caches it in a local SQLite database, and uses OpenAI to generate AI-driven insights. It includes a Gradio-based chat interface that allows users to explore crypto coins and receive text, image, or audio responses in real time.

In [1]:
# imports

import os
import json
import time
import sqlite3
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [19]:
# Initialization

load_dotenv(override=True)

class Config:
    pass

config = Config()

REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "COINGECKO_PUBLIC_URL"]

for var in REQUIRED_ENV_VARS:
    value = os.getenv(var)
    if not value:
        raise RuntimeError(f"{var} not set")
    setattr(config, var, value)

MODEL = "gpt-4.1-mini"
openai = OpenAI()
client = OpenAI(api_key=config.OPENAI_API_KEY)

DB = "coinscan.db"



In [3]:
# Terminal logging
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

logger = logging.getLogger("CoinScanBot")


In [4]:
#DB setup

def init_db():
    conn = sqlite3.connect(DB)
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS coin_metrics (
            coin_id TEXT PRIMARY KEY,
            market_cap REAL,
            holders INTEGER,
            popularity REAL,
            last_updated INTEGER
        )
    """)

    conn.commit()
    conn.close()

init_db()


In [5]:
# CRUD for coin metrics 

CACHE_TTL = 600  # 10 minutes

def get_cached_metrics(coin_id):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()

    cur.execute("""
        SELECT market_cap, holders, popularity, last_updated
        FROM coin_metrics
        WHERE coin_id=?
    """, (coin_id,))
    row = cur.fetchone()
    conn.close()

    if not row:
        logger.info(f"[CACHE] MISS — {coin_id}")
        return None

    market_cap, holders, popularity, ts = row

    if time.time() - ts > CACHE_TTL:
        logger.info(f"[CACHE] STALE — {coin_id}")
        return None

    logger.info(f"[CACHE] HIT — {coin_id}")
    return {
        "market_cap": market_cap,
        "holders": holders,
        "popularity": popularity
    }



def save_metrics(coin_id, market_cap, holders, popularity):
    conn = sqlite3.connect(DB)
    cur = conn.cursor()

    cur.execute("""
        INSERT OR REPLACE INTO coin_metrics
        VALUES (?, ?, ?, ?, ?)
    """, (coin_id, market_cap, holders, popularity, int(time.time())))

    conn.commit()
    conn.close()


In [8]:
# Fetch coin data from Coingecko

def fetch_coin_data(coin_id):
    cached = get_cached_metrics(coin_id)
    if cached:
        return cached

    logger.info(f"[API] Fetching data from CoinGecko for {coin_id}")

    url = f"{config.COINGECKO_PUBLIC_URL}{coin_id}"
    r = requests.get(url, timeout=10)
    data = r.json()

    market_cap = data["market_data"]["market_cap"]["usd"]
    holders = data.get("community_data", {}).get("reddit_subscribers", 0)
    popularity = data.get("community_score", 0)

    save_metrics(coin_id, market_cap, holders, popularity)

    return {
        "market_cap": market_cap,
        "holders": holders,
        "popularity": popularity
    }



In [6]:

# Tool schema definitions

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_cached_metrics",
            "description": "Get cached crypto metrics if available",
            "parameters": {
                "type": "object",
                "properties": {
                    "coin_id": {"type": "string"}
                },
                "required": ["coin_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "fetch_coin_data",
            "description": "Fetch crypto metrics from CoinGecko",
            "parameters": {
                "type": "object",
                "properties": {
                    "coin_id": {"type": "string"}
                },
                "required": ["coin_id"]
            }
        }
    }
]


In [9]:
# Build the tool data

TOOL_REGISTRY = {
    "get_cached_metrics": get_cached_metrics,
    "fetch_coin_data": fetch_coin_data,
}

def handle_tool_calls(message):
    responses = []

    for call in message.tool_calls:
        tool_name = call.function.name
        args = json.loads(call.function.arguments)

        logger.info(f"[TOOL CALL] {tool_name}({args})")

        tool_fn = TOOL_REGISTRY.get(tool_name)
        result = tool_fn(**args) if tool_fn else {"error": "Unknown tool"}

        logger.info(f"[TOOL RESULT] {tool_name} → {result}")

        responses.append({
            "role": "tool",
            "tool_call_id": call.id,
            "content": json.dumps(result)
        })

    return responses



In [10]:
# System prompt
system_message = """
You are CoinScanBot, a crypto due diligence assistant.

Rules:
- Always use tools to gather objective data
- Never give financial advice
- Never say buy or sell
- Focus on risks, transparency, and maturity
- Clearly list red flags and positive signals
"""


In [ ]:
import base64
from PIL import Image
from io import BytesIO

def generate_coin_image(coin_name: str, summary):
    logger.info("🎨 Generating coin image")

    prompt = f"""
        Create a clean infographic-style illustration representing the crypto project {coin_name}.
        Reflect its risk profile and maturity:
        {summary}
    """

    img = client.images.generate(
        model="gpt-image-1", # needs account verification!
        prompt=prompt,
        size="1024x1024"
    )

    image_base64 = img.data[0].b64_json
    image_bytes = base64.b64decode(image_base64)

    image = Image.open(BytesIO(image_bytes))
    return image


In [ ]:
def generate_audio(text: str) -> str:
    logger.info("🔊 Generating audio narration")

    audio_path = "analysis.mp3"

    with client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=text,
    ) as response:

        response.stream_to_file(audio_path)

    return audio_path


In [13]:
# Main chat function

def chat(message, history):
    messages = (
        [{"role": "system", "content": system_message}]
        + history
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    while response.choices[0].finish_reason == "tool_calls":
        logger.info("[LLM] Tool call requested")

        msg = response.choices[0].message
        tool_responses = handle_tool_calls(msg)

        messages.append(msg)
        messages.extend(tool_responses)

        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

    assistant_reply = response.choices[0].message.content

    history.append({
        "role": "assistant",
        "content": assistant_reply
    })

    return history



In [14]:
def generate_multimodal_outputs(history):
    last_answer = history[-1]["content"]

    image_url = generate_coin_image("Coin", last_answer)
    audio_path = generate_audio(last_answer)

    return image_url, audio_path


In [ ]:
# Build the chatbot with Gradio

def put_message_in_chatbot(message, history):
    history.append({
        "role": "user",
        "content": message
    })
    return "", history

with gr.Blocks() as ui:
    gr.Markdown("# 🪙 CoinScanBot – Multimodal Crypto Analysis")

    with gr.Row():
        chatbot = gr.Chatbot(type="messages", height=400)

    with gr.Row():
        msg = gr.Textbox(label="Ask about a crypto project")

    with gr.Row():
        image_output = gr.Image(label="Visual Analysis")
        audio_output = gr.Audio(label="Spoken Summary")

    msg.submit(
        put_message_in_chatbot,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    ).then(
        chat,
        inputs=[msg, chatbot],
        outputs=chatbot
    ).then(
        generate_multimodal_outputs,
        inputs=chatbot,
        outputs=[image_output, audio_output]
    )

ui.launch(inbrowser=True)

